In [1]:
from queue import PriorityQueue

In [2]:
valves = {}
flow_rates = {}
start_valve = None
with open('input.txt', 'r') as fd:
# with open('example_input.txt', 'r') as fd:
    for line in fd:
        parts = line.replace('=', ' ').replace(';', '').replace(',', '').split()
        node = parts[1]
        rate = int(parts[5])
        children = parts[parts.index('to') + 2:]
        if start_valve is None:
            start_valve = node

        valves[node] = children
        flow_rates[node] = rate

    

In [ ]:
def calculate_distances(valves):
    all_distances = {}

    for valve in valves:
        distances = {}
        distances[valve] = 0

        visited = set()
        unvisited = PriorityQueue()
        unvisited.put((0, valve))
        while not unvisited.empty():
            p, cell = unvisited.get()
            for neighbor in valves[cell][1]:
                if neighbor not in visited:
                    new_dist = distances[cell] + 1
                    if new_dist < distances.get(neighbor, 1e10):
                        distances[neighbor] = new_dist
                        unvisited.put((new_dist, neighbor))

            visited.add(cell)
        
        all_distances[valve] = distances
    return all_distances

In [ ]:
dists = calculate_distances(valves)

In [ ]:
graph = {
    0: set([1, 2]),
    1: set([3]),
    2: set([4, 5]),
    3: set([2]),
    4: set([1]),
    5: set([0])
}

In [3]:
def shortest_paths(graph, start):
    visited = set()
    distances = {}

    q = PriorityQueue()
    q.put((0, start))
    
    while not q.empty():
        node_cost, node = q.get()
        for nn in graph[node]:
            if nn not in visited:
                cost = node_cost + 1
                if nn not in distances or cost < distances[nn]:
                    distances[nn] = cost
                    q.put((cost, nn))
        
        visited.add(node)

    return distances

In [4]:
non_zero_valves = {k: v for k, v in valves.items() if flow_rates[k] > 0}
non_zero_valve_distances = {}
for valve in list(non_zero_valves) + ['AA']:
    ds = shortest_paths(valves, valve)
    non_zero_valve_distances[valve] = {k: v for k, v in ds.items() if k in non_zero_valves}

#### Part 1 

In [ ]:
def part_1(distances, start, minutes, opened):
    opened = set(opened)
    stack = [(start, [], minutes, opened, 0)]
    best = 0
    best_path = None
    while stack:
        node, path, minutes, opened, score = stack.pop()
        # node = path[-1]
        to_visit = set(distances[node]) - opened
        if not to_visit:
            if score > best:
                best_path = path.copy()
                best = score
        for nn in to_visit:
            dist = distances[node][nn]
            new_minutes = minutes - dist - 1
            if new_minutes <= 0:
                # best = max(score, best)
                if score > best:
                    best = score
                    best_path = path.copy()
            else:
                new_path = path + [nn]
                new_opened = opened.copy()
                new_opened.add(nn)
                new_score = score + new_minutes * flow_rates[nn]
                stack.append((nn, new_path, new_minutes, new_opened, new_score))

    return best, best_path

#### Part 2

In [5]:
from itertools import combinations

In [ ]:
results = {}
combos = list(combinations(non_zero_valves, 7))
num_combos = len(combos)
for i, opened in enumerate(combos):
    print(f'{i} / {num_combos}', end='\r')
    results[opened] = part_1(non_zero_valve_distances, 'AA', 26, opened)

In [ ]:
totals = []
num_results = len(results)
for i, (my_score, my_path) in enumerate(results.values()):
    print(f'{i} / {num_results}', end='\r')
    el_score, el_path = part_1(non_zero_valve_distances, 'AA', 26, my_path)
    totals.append(
        ((tuple(my_path), tuple(el_path)), my_score + el_score)
    )

In [ ]:
sorted(totals, key=lambda x: x[1], reverse=True)[0]

In [6]:
def compute_all_traversals(distances, start, minutes, flow_rates):
    stack = [(start, minutes, set(), 0)]
    scores = {}
    while stack:
        node, minutes, opened, score = stack.pop()

        # The order that the valves were visited doesn't matter for finding the
        # optimal route. We just want the highest score for any permutation of
        # each set of valves.
        _opened = frozenset(opened)
        scores[_opened] = max(scores.get(_opened, 0), score)

        # Don't visit valves we have already opened
        to_visit = set(distances[node]).difference(opened)
        for nn in to_visit:
            dist = distances[node][nn]
            new_minutes = minutes - dist - 1
            if new_minutes > 0:
                # We have time to travel to and open this valve
                new_score = score + new_minutes * flow_rates[nn]
                stack.append(
                    (nn, new_minutes, opened | {nn}, new_score)
                    )

    return scores

In [7]:
scores = compute_all_traversals(non_zero_valve_distances, 'AA', 26, flow_rates)
sorted(scores.values())[-1]

1490

In [ ]:
scores = {set()}

In [8]:
# Find largest score for two traversals with non-overlapping paths
totals = {}
for path1, score1 in scores.items():
    for path2, score2 in scores.items():
        if path1 and path2 and not path1.intersection(path2):
            totals[(path1, path2)] = score1 + score2

sorted(totals.items(), key=lambda x: x[1], reverse=True)[0]

((frozenset({'BV', 'CF', 'EZ', 'JX', 'LB', 'WM'}),
  frozenset({'HG', 'IK', 'IT', 'VK', 'WH', 'WL'})),
 2469)

#### Other

In [ ]:
sorted(paths, key=lambda x: x[0], reverse=True)

In [ ]:
all_paths = []

def traverse(graph, node, visited):
    visited.append(node)
    if not graph[node]:
        all_paths.append(visited)
    else:
        for node in graph[node]:
            if node not in visited:
                traverse(graph, node, visited.copy())

def dfs_search_all_paths(graph, node, steps):
    stack = [([node], steps, [])]
    while stack:
        path, steps, visited = stack.pop()
        children = graph[path[-1]][1]
        # Visited all paths, no time left, or no child nodes
        if len(set(path)) == len(graph) or steps == 0 or not children:
            yield path
        else:
            for nn in children:
                stack.append((path + [nn], steps - 1, visited))

list(dfs_search_all_paths(valves, 'AA', 6))

In [ ]:
[[0]]
[[0, 1], [0, 2]]
[[0, 1, 3], [0, 2, 4], [0, 2, 5]]